In [1]:
import os
import sys
import numpy as np
sys.path.insert(0, os.path.dirname(os.getcwd()))
import tensorflow as tf
import numpy as np
from waymo_open_dataset import dataset_pb2 as open_dataset
from utils import lidar_crop, save_darknet, label_in_polygon

2022-07-22 22:36:29.568192: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib64:
2022-07-22 22:36:29.568210: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
TOO_DARK_THRESHOLD = 50
SCALED_WIDTH = 800
SCALED_HEIGHT = 533
LABEL_VEHICLE = 1
LABEL_PEDESTRIAN = 2
LABEL_CYCLIST = 4
SUPPORTED_LABELS = {
    LABEL_VEHICLE: 0,
    LABEL_PEDESTRIAN: 1,
    LABEL_CYCLIST: 2
}

In [10]:
training_files = ['Data/unprocessed/training/' + file for file in os.listdir('Data/unprocessed/training')]
dataset = tf.data.TFRecordDataset(training_files, buffer_size=tf.constant(int(pow(10,6)), tf.int64), num_parallel_reads=16)
OUTPUT_PATH = 'Data/darknet-data'
img_id = len(os.listdir(f'{OUTPUT_PATH}/obj')) // 2

training_index = open('Data/darknet-data/training.txt', 'w')
print('Processing training data')
nb_pedestrians = 0
MAX_PEDESTRIANS = 20000
for data in dataset:
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))

    label_types = [camera_labels.labels for camera_labels in frame.camera_labels if camera_labels.name == 1][0]
    label_types = [label.type for label in label_types if label.box.length > 50 and label.box.length > 50 and label.type in SUPPORTED_LABELS.keys()]

    if LABEL_PEDESTRIAN in label_types:
        nb_pedestrians += 1

    if (nb_pedestrians <= MAX_PEDESTRIANS or LABEL_PEDESTRIAN not in label_types) and LABEL_CYCLIST not in label_types:
        continue

    image = frame.images[0]
    image = np.array(tf.io.decode_jpeg(image.image))

    if np.mean(image) < TOO_DARK_THRESHOLD:
        continue
    
    og_width = image.shape[1]
    og_height = image.shape[0]
    (image, labels) = lidar_crop(frame, image)
    labels = [label for label in labels if label_in_polygon(label, image, 0.25)]
    
    save_darknet(image, labels, OUTPUT_PATH, og_width, og_height, training_index, img_id)
    
    img_id += 1
    if img_id % 100 == 0:
        print(img_id)

training_index.close()

Processing training data


/home/antoinebrassardlahey/MTI830/utils.py:174: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  exteriors = [int_coords(poly.exterior.coords) for poly in alpha_shape]


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [3]:
validation_files = ['Data/unprocessed/validation/' + file for file in os.listdir('Data/unprocessed/validation')]
dataset = tf.data.TFRecordDataset(validation_files, buffer_size=tf.constant(int(pow(10,6)), tf.int64), num_parallel_reads=16)
OUTPUT_PATH = 'Data/darknet-data2'

validation_index = open('Data/darknet-data2/validation.txt', 'w')
img_id = len(os.listdir(f'{OUTPUT_PATH}/obj')) // 2
nb_pedestrians = 0
MAX_PEDESTRIANS = 20000

print('Processing validation data')
for data in dataset:
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))

    label_types = [camera_labels.labels for camera_labels in frame.camera_labels if camera_labels.name == 1][0]
    label_types = [label.type for label in label_types if label.box.length > 50 and label.box.length > 50 and label.type in SUPPORTED_LABELS.keys()]

    if LABEL_PEDESTRIAN not in label_types and LABEL_CYCLIST not in label_types:
        continue

    image = frame.images[0]
    image = np.array(tf.io.decode_jpeg(image.image))

    if np.mean(image) < TOO_DARK_THRESHOLD:
        continue
    
    og_width = image.shape[1]
    og_height = image.shape[0]
    
    labels = [camera_labels.labels for camera_labels in frame.camera_labels if camera_labels.name == 1][0]
    labels = [label for label in labels if label.box.length > 50 and label.box.length > 50 and label.type in SUPPORTED_LABELS.keys()]

    save_darknet(image, labels, OUTPUT_PATH, og_width, og_height, validation_index, img_id)
    
    img_id += 1
    if img_id % 100 == 0:
        print(img_id)

validation_index.close()

2022-07-22 22:36:34.589860: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-22 22:36:34.590125: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/antoinebrassardlahey/anaconda3/envs/mti830-3d/lib/python3.8/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:
2022-07-22 22:36:34.590175: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/antoinebrassardlahey/anaconda3/envs/mti830-3d/lib/python3.8/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:
2022-0

Processing validation data
20500
20600
20700
20800
20900
21000
21100
21200
21300
21400
21500
21600
21700
21800
21900
22000
22100
22200
22300
22400
22500
22600
22700
22800
22900
23000
23100
23200
23300
23400
23500
23600
23700
23800
23900
24000
24100
24200
24300
24400
24500
24600
24700
24800
24900
25000
25100
25200
25300
25400
25500
25600
25700
25800
25900
26000
26100
26200
26300
26400
26500
26600
26700
26800
26900
27000
27100
27200
27300
27400
27500
27600
27700
27800
27900
28000
28100
28200
28300
28400
28500
28600
28700
28800
28900
29000
29100
